In [2]:
import numpy as np
import pygame as pg

pygame 2.1.3.dev8 (SDL 2.0.22, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


# <u> A. Implémentation du jeu : </u>

On propose de tester les algorithmes et différentes versions sur un jeu plus simple : le `snake`. Le principe du jeu est le suivant : un serpent (le snake) se déplace sur une grille à la recherche des pommes, il doit en manger le plus possible, mais sans toucher ni les murs, ni son propre corps, sans quoi il meurt. A chaque pomme mangé, il grandit de $1$, augmentant la difficulté du jeu. Le serpent peut se déplacer sur toute la grille en allant tout droit, à sa gauche ou à sa droite par rapport à sa direction. Il ne peut donc pas aller en bas, s'il est dirigé vers le haut, ou ne peut pas aller à gauche s'il est dirigé vers la droite par exemple. 

On implémente en définissant une classe `Snake_environnement` et les méthodes suivantes :  

<u> a. </u> Le jeu est initialisé par des paramètres initiaux (taille de la grille, taille initiale du serpent, gain, pénalité...) correspondant à la fonction `__init__` de notre `Snake`. Une fois les constantes du jeu décidées, on définit la grille de jeux et positionne le serpent (le plus au milieu possible et dirigé vers le haut) sur celle-ci, on position une première pomme sur l'écran. 
On décide comme convention que la carte sera notée d'un $0$ si la case est libre, d'un $1$ si elle est occupée par une pomme, et $0.5$ si c'est par le serpent. 
Pour connaître la direction dans laquelle se dirige le serpent on garde en mémoire le dernier mouvement : 
<font color=gray>
    en effet, si le serpent vient d'avancer vers une direction, c'est qu'il est dirigé à présent dans cette direction
    </font>. La position du serpent est implémenté par une liste dont la longueur correspond à la taille du serpent et la tête est située au début de la liste. 
    
<u> b. </u> Le placement des pommes est aléatoire sur la grille et sera implémenté par la fonction `new_apple`.

<u> c. </u> Lorsque le serpent se déplace, la méthode `snake_move` met à jour la position du serpent en fonction de la prochaine position.

<u> d.</u> Maintenant, le principe du jeu est encodé, mais il est nécessaire de faire une fonction qui permet au jeu d'avancer. Cette fonction `step` permet de faire avancer le jeu d'une étape en fonction de l'action fournit par l'agent. Selon l'action effectué, la fonction permet de déplacer le serpent, calculer les gains, vérifier que le serpent n'est pas dans une situation de Game Over.

<u> e. </u> Enfin, pour terminer, il faut une fonction qui réinitialise le jeu, pour chaque début de partie. Ceci est implémenter dans `reset`. 

In [1]:

class Snake_environment: 
    def __init__(self): 
        self.row = 8                    # Nombre de lignes
        self.col = 8                    # Nombre de colonnes
        self.snake_initial = 2
        self.time_penalty = -3          # Penalité de temps
        self.death_penalty = -100       # Pénalité de mort
        self.apple_reward = 200         # Gain

        self.snake_position = []
        
        #Grille de jeu : 0 = rien, 0.5 = serpent, 1 = pomme
        self.map =  np.zeros((self.row, self.col))

        for i in range(self.snake_initial):  #Taille initiale d'un snake : self.snake_initial 
            row_init = int(self.row/2)+i
            col_init = int(self.col/2)
            self.snake_position+=[(row_init,col_init)]
            self.map[row_init,col_init] = 0.5

        self.apple_position = self.new_apple()

        self.collected = False
        self.last_action = 0

    def new_apple(self):
        new_row = np.random.randint(0,self.row)
        new_col = np.random.randint(0,self.col)

        while self.map[new_row,new_col]==0.5 : # i.e. le serpent est dessus
            new_row = np.random.randint(0,self.row)
            new_col = np.random.randint(0,self.col)

        self.map[new_row,new_col] = 1  # On place la pomme sur la grille
        return(new_row,new_col)

    def snake_move(self,next_row,next_col,apple_collected):
        self.snake_position.insert(0,(next_row,next_col))

        if not apple_collected :
            self.snake_position.pop()
        # Sinon le serpent grandit de 1, donc on ne change rien

        elif apple_collected :
            self.collected = True

        # On redéfini la grille :
        self.map = np.zeros((self.row,self.col))

        for i in range(len(self.snake_position)):  #len(self.snake_position)= taille du serpent + 1
            pos_row = self.snake_position[i][0]
            pos_col = self.snake_position[i][1]
            self.map[pos_row,pos_col]=0.5

        apple_pos_row = self.apple_position[0]
        apple_pos_col = self.apple_position[1]
        self.map[apple_pos_row,apple_pos_col]= 1

    def step(self,action):
        '''
        Notre convention : 
        ACTION_UP = 0
        ACTION_DOWN = 1
        ACTION_RIGHT = 2
        ACTION_LEFT = 3
        '''
        #de base, on réinitialise ça : (en premier si jamais il rencontre une pomme ensuite)
        self.collected = False

        # Il faut d'abord vérifier que la nouvelle action est réalisable, dans le cas contraire, le serpent avance de 1 dans sa direction
        # Les différents mouvements irréalisable sont :
        #    - aller à gauche après être aller à droite => l'action réalisée sera alors aller à droite
        #    - aller en haut après être aller en bas => l'action réalisée sera alors aller en bas
        #    - aller à droite après être aller à gauche => l'action réalisée sera alors aller à gauche
        #    - aller en bas après être aller en haut => l'action réalisée sera alors aller en haut
        if   action == 3 and self.last_action == 2 :
            action = 2
        elif action == 0 and self.last_action == 1 :
            action = 1
        elif action == 2 and self.last_action == 3 :
            action = 3
        elif action == 1 and self.last_action == 0 :
            action = 0

        # On effectue l'action souhaitée, regarde le Rt associé et effectue une étape
        pos_row = self.snake_position[0][0]
        pos_col = self.snake_position[0][1]

        if action == 0 :
            next_row = pos_row-1
            next_col = pos_col
            
            #Le serpent perd s'il rencontre un mur ou lui-même:
            #Grille de jeu : 0 = rien, 0.5 = serpent, 1 = pomme
            if next_row < 0 :
                game_over = True
                reward = self.death_penalty
            else:
                if self.map[next_row,next_col] == 0.5:
                    game_over = True
                    reward = self.death_penalty
                if self.map[next_row,next_col] == 1:
                    game_over = False
                    reward = self.apple_reward
                    self.snake_move(next_row,next_col, apple_collected=True)
                if self.map[next_row,next_col] == 0:
                    game_over = False
                    reward = self.time_penalty
                    self.snake_move(next_row,next_col, apple_collected = False)

        elif action == 1 :
            next_row = pos_row+1
            next_col = pos_col
            if next_row > self.row -1 :
                game_over = True
                reward = self.death_penalty
            else:
                if self.map[next_row,next_col] == 0.5:
                    game_over = True
                    reward = self.death_penalty
                if self.map[next_row,next_col] == 1:
                    game_over = False
                    reward = self.apple_reward
                    self.snake_move(next_row,next_col, apple_collected=True)
                if self.map[next_row,next_col] == 0:
                    game_over = False
                    reward = self.time_penalty
                    self.snake_move(next_row,next_col, apple_collected = False)

        elif action == 2 :
            next_row = pos_row
            next_col = pos_col+1
            if next_col > self.col -1 :
                game_over = True
                reward = self.death_penalty
            else:
                if self.map[next_row,next_col] == 0.5:
                    game_over = True
                    reward = self.death_penalty
                if self.map[next_row,next_col] == 1:
                    game_over = False
                    reward = self.apple_reward
                    self.snake_move(next_row,next_col, apple_collected=True)
                if self.map[next_row,next_col] == 0:
                    game_over = False
                    reward = self.time_penalty
                    self.snake_move(next_row,next_col, apple_collected = False)
        elif action == 3 :
            next_row = pos_row
            next_col = pos_col-1
            if next_col < 0 :
                game_over = True
                reward = self.death_penalty
            else:
                if self.map[next_row,next_col] == 0.5:
                    game_over = True
                    reward = self.death_penalty
                if self.map[next_row,next_col] == 1:
                    game_over = False
                    reward = self.apple_reward
                    self.snake_move(next_row,next_col, apple_collected=True)
                if self.map[next_row,next_col] == 0:
                    game_over = False
                    reward = self.time_penalty
                    self.snake_move(next_row,next_col, apple_collected = False)

        # On reset les paramètres du jeu correctement (seulement maintenant, car aurait pu changer du choix initial)
        self.last_action = action

        return(self.map,reward,game_over)

    def reset(self):
        self.map = np.zeros((self.row,self.col))
        self.snake_position = []

        for i in range(self.snake_initial):
            row_init = int(self.row/2)+i
            col_init = int(self.col/2)

            self.snake_position+=[(row_init,col_init)]

            self.map[row_init,col_init] = 0.5

        apple_pos_row = self.apple_position[0]
        apple_pos_col = self.apple_position[1]
        self.map[apple_pos_row,apple_pos_col]= 1
    
        self.last_move = 0